# Driving Behavior Classification

## Import Necessary Libraries

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
import pickle
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler

## Import necessary API
import sys
sys.path.append('../../../../')
from api.v2.util.data_load import data_load
from api.v2.Preprocessing.Make_Time_Feature import TimeFeatureGenerator
from api.v2.model.TCN import TCN

## Import libraries for the model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results 
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Base Parameter Setting
* Set parameters based on the information identified during EDA (Exploratory Data Analysis).

In [2]:
# Set Driving Behavior Data Base Parameter

# Set MachBase Neo URL address
URL = 'http://127.0.0.1:5654'
# Set Tag Table Name
table = 'driving_behavior'
# Select Tag Name -> Can Check Tag Names Using command 'show_column(URL, table)'
tags = ['AccX', 'AccY', 'AccZ', 'Class', 'GyroX', 'GyroY', 'GyroZ']
# Wrap each item in the list with single quotes and separate with commas
tags_ = ",".join(f"'{tag}'" for tag in tags)
# Set Tag Name
name = quote(tags_, safe=":/")
# Set resample Option -> D(day), H(hour), T(minute), S(second)
resample_freq = None
# Set Start time
start_time = '2025-07-18 00:00:00'
# Set End time 
end_time = '2025-07-18 01:52:07'
# Set TimeFormat - > 'default' or quote('2006-01-02 15:04:05.000000')(Divided down to the nanosecond)
timeformat = 'default'

## Driving Behavior Data Load

In [3]:
# Driving Behavior Data Load
df = data_load(URL, table, name, start_time, end_time, timeformat, resample_freq)

# Rename the 'Class' column to 'label'
df.rename(columns={'Class': 'label'}, inplace=True)

# Move the 'label' column to the last position
df = df.reindex(columns=[col for col in df.columns if col != 'label'] + ['label'])

# Convert the 'label' column to integer type
df['label'] = df['label'].astype(int)

In [4]:
df

NAME,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,label
TIME,,,,,,,
2025-07-18 00:00:00,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,0
2025-07-18 00:00:01,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,0
2025-07-18 00:00:02,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,0
2025-07-18 00:00:03,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,0
2025-07-18 00:00:04,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,0
...,...,...,...,...,...,...,...
2025-07-18 01:52:03,-0.713858,-0.652975,-0.164015,-0.147829,-1.309466,0.517250,1
2025-07-18 01:52:04,1.514261,0.330070,1.020714,1.321302,1.707598,-0.674548,1
2025-07-18 01:52:05,1.280216,-1.735172,-2.332695,0.583376,0.690507,-0.468075,1


In [5]:
# Split the data into train, test sets
train = df.iloc[:3728]
test = df.iloc[3728:]

## Data Preprocessing
* 1 Min-Max Scaling
* 2 Make Time Feature

In [6]:
# Set up scalers
scaler = MinMaxScaler()

# Apply scalers
train_ = scaler.fit_transform(train.iloc[:, :-1].values)
test_ = scaler.transform(test.iloc[:, :-1].values)

# Set each DataFrames
train_scaled = pd.DataFrame(train_, index=train.index)  
train_scaled['label'] = train['label'].values

test_scaled = pd.DataFrame(test_, index=test.index)
test_scaled['label'] = test['label'].values

# Save Scaler
with open('./result/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [7]:
# Set TimeFeatureGenerator
feature_generator = TimeFeatureGenerator()
 
# Make Time Featrue
train_time_feature = feature_generator.generate_features(train.index)
test_time_feature = feature_generator.generate_features(test.index)

# concat origin dataset
train_scaled = pd.concat([train_scaled, train_time_feature], axis=1)
test_scaled = pd.concat([test_scaled, test_time_feature], axis=1)

## Dataset & Loader Setup

In [8]:
class Driving_Dataset(Dataset):
    def __init__(self, data, target_column, seq_length=10):
        self.data = data
        self.target_column = target_column
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length + 1

    def __getitem__(self, idx):
        # Input data: Values of multiple variables over seq_length - 1
        x = self.data[idx:idx + self.seq_length].drop(columns=[self.target_column]).values.reshape(-1, self.seq_length)  # Excluding target column
        # Target data: Target variable value on the seq_length 
        y = self.data[self.target_column].iloc[idx + self.seq_length - 1]
        return x, y

In [9]:
# Set up datasets  
train_ = Driving_Dataset(train_scaled, target_column='label', seq_length=2)
test_ = Driving_Dataset(test_scaled, target_column='label', seq_length=2)

# Set up data loaders
train_dataloader = DataLoader(train_, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_, batch_size=64, shuffle=False)

In [10]:
# Verify DataLoader application and check the shape of the input data
print(list(train_dataloader)[0][0].shape)

torch.Size([64, 16, 2])


## Model Configuration
* Using Temporal Convolutional Network(TCN) model.

In [11]:
# Model configuration parameters
input_channels = 16
out_channels = 3
hidden_channels = [32, 64] 
kernel_size = 1
dropout = 0.05

# Learning rate
learning_rate = 0.001

# Initialize the model, loss, and optimizer
model = TCN(num_inputs=input_channels, num_channels=hidden_channels,out_channel=out_channels, kernel_size=kernel_size, dropout=dropout).to(device)

# Configure loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check the model architecture
print(model)

TCN(
  (network): Sequential(
    (0): TemporalBlock(
      (conv1): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
      (relu1): ReLU()
      (dropout1): Dropout(p=0.05, inplace=False)
      (conv2): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      (relu2): ReLU()
      (dropout2): Dropout(p=0.05, inplace=False)
      (net): Sequential(
        (0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
        (1): ReLU()
        (2): Dropout(p=0.05, inplace=False)
        (3): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (4): ReLU()
        (5): Dropout(p=0.05, inplace=False)
      )
      (downsample): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
    )
    (1): TemporalBlock(
      (conv1): Conv1d(32, 64, kernel_size=(1,), stride=(1,), dilation=(2,))
      (relu1): ReLU()
      (dropout1): Dropout(p=0.05, inplace=False)
      (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), dilation=(2,))
      (relu2): ReLU()
      (dropout2): Dropout(p=0.05, inplace

## Model Training

* Save the model with the F1 Score.

In [12]:
# Initialize training loss
train_loss = []
# Initialize training accuracy
train_acc = []
# Initialize total step
total_step = len(train_dataloader)
# Set number of epochs
epoch_in = trange(20, desc='training')
# Initialize best F1 Score value
best_f1= 0

# Start model training
for epoch in epoch_in:

    model.train()
    running_loss = 0.0
    correct = 0
    total=0
    preds_ = []
    targets_ = []
    
    # Initialize loss
    train_loss = []
    
    for batch_idx, (data, target) in enumerate(train_dataloader):
        
        data = data.to(device).float()
        target = target.to(device).long().squeeze()
        
        optimizer.zero_grad()
        
        # Input to the model
        outputs = model(data)
        
        # Calculate loss
        loss = criterion(outputs.squeeze(), target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Set label predictions 
        _,pred = torch.max(outputs.squeeze(), dim=1)
        target_ = target.view_as(pred)
        correct += torch.sum(pred==target).item()
        total += target.size(0)
        
        preds_.append(pred)
        targets_.append(target_)
            
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')

    # Combine predictions and labels collected from all batches
    preds_ = torch.cat(preds_).detach().cpu().numpy()
    targets_ = torch.cat(targets_).detach().cpu().numpy()
    
    f1score = f1_score(targets_, preds_,  average='macro')
    if best_f1 < f1score:
        best_f1 = f1score
        # Save the best model 
        with open("./result/Driving_Behavior_full.txt", "a") as text_file:
            print('epoch=====',epoch, file=text_file)
            print(classification_report(targets_, preds_, digits=4), file=text_file)
        print('model save')
        torch.save(model, f'./result/Driving_Behavior_full.pt') 
    epoch_in.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")

training:   0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\co


train loss: 1.2593444353443082, train acc: 32.1706
model save

train loss: 1.0654736737073478, train acc: 45.9082
model save

train loss: 0.9132415241861748, train acc: 58.5994
model save

train loss: 0.7818744924882333, train acc: 61.5508
model save

train loss: 0.7358570639537332, train acc: 61.7387
model save

train loss: 0.6946969108766544, train acc: 67.2122
model save

train loss: 0.5490076067301018, train acc: 72.0955
model save

train loss: 0.6034766961840154, train acc: 72.8736
model save

train loss: 0.5163715459288974, train acc: 77.7837
model save

train loss: 0.6221765915081852, train acc: 76.0397

train loss: 0.46710281407082466, train acc: 81.0035
model save

train loss: 0.619547666272472, train acc: 79.1790

train loss: 0.41201301466913964, train acc: 84.2232
model save

train loss: 0.6172712384427156, train acc: 79.4204

train loss: 0.38820596942739444, train acc: 85.6990
model save

train loss: 0.5746732959109437, train acc: 84.2769

train loss: 0.38212503441619927, 

## Model Testing

In [13]:
# Load the best model
model_ = torch.load(f'./result/Driving_Behavior_full.pt')

In [14]:
# Model testing
preds_test = []
target_test = []
with torch.no_grad():
    model_.eval()
    for batch_idx, (data, target) in enumerate(test_dataloader):
        data = data.to(device).float()
        target = target.to(device).long()
        
        outputs_t = model_(data)
        
        _,pred_t = torch.max(outputs_t, dim=1)
        targets_t = target.view_as(pred_t).to(device)

        preds_test.append(pred_t)
        target_test.append(targets_t)
        
    # Combine predictions and labels collected from all batches
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


## Model Performance Evaluation

In [16]:
print(classification_report(target_test, preds_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       997
           1       1.00      0.89      0.94      1273
           2       0.15      0.28      0.19       729

    accuracy                           0.44      2999
   macro avg       0.38      0.39      0.38      2999
weighted avg       0.46      0.44      0.45      2999

